In [105]:
from ACLScheduleTest.Simulate import Simulator
import numpy as np

In [126]:
Print_limit = 1
Epoch  = 0
Out_Epoch = 0
from dill.source import getsource
def schedule_in_func(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
    global  Print_limit, Epoch
    Epoch += 1
    if Print_limit >= Epoch:
        print('+'*60)

        print(f'in_func Info: Epoch {Epoch}\n')
        print(f'workers:调度其中各个workers的状态, 是一个词典(dict)')
        print('key是worker的名字，如\'worker_0\'; ')
        print('value有两种，如果该worker没有正在运行, 就输出None；如果正在计算某个节点node并且已经运行了1单位时间，则输出[\'node\', 1].\n 注意，在题目假设下，每个任务都以单位时间执行，因此这里只有可能出现0')
        for key in workers.keys():
            print(key, workers[key])


        print(f'\ncache:内存存储状态，存储各个节点内用于计算的数值。也是一个dict')
        print('key是节点node的名字，value是存储的具体数值')
        for key in cache.keys():
            print(key, cache[key])
        
        

        print(f'\nav_mem:当前状态下可用的内存，float格式\n{av_mem}')
        
        print(f'\nfactor_node_dict:因子节点的状态，key是node名字，value是输出数值，如果是None表示未被计算到')
        for key in factor_node_dict.keys():
            print(key, factor_node_dict[key])

        print(f'\ncalculated_node_dict:记录节点是否已经被计算完，value为bool格式. 在demo中未被用到\n')
        for key in calculated_node_dict.keys():
            print(key, calculated_node_dict[key])

        print(f'\ndsk:网络结构，运算的依赖关系\n')
        print('每个key是node的名字， value是一个元组，第一个值是要计算的lambda函数，后面为自变量')
        print('自变量可以是要计算的数值，也可以是其他node。')
        for key in dsk.keys():
            print(key, getsource(dsk[key][0].func), dsk[key][1:])

        print(f'\nglobal_dict在demo中未被用到。保留给选手可以自定义的参数。:\n{global_dict}')
        

        matrix = np.zeros((100,100))    
        nodeList = list(dsk.keys())
        for idx, node in enumerate(nodeList):
            for n in dsk[node][1:]:
                if n in nodeList:
                    id = nodeList.index(n)
                    matrix[idx, id]=1
        print(matrix)
        sum_next, sum_pre = matrix.sum(axis=0), matrix.sum(axis=1)
        print(sum_next)
        print(sum_pre)
        
    # 检查因子节点是否还有未计算的（None）。如果没有，表明所有因子节点都有计算结果，此时返回None并退出计算
    factor_node = set([k for k in factor_node_dict if factor_node_dict[k] is None])

    if len(factor_node) == 0:
        return None

    # 检查正在工作的worker，并将这些worker的正在计算的节点组成集合记为workers_nodes
    workers_nodes = set([workers[k][0] for k in workers if workers[k] is not None])

    def topoOrder():
        topoQueue = []
        q = list(factor_node - set(cache.keys()) - workers_nodes)

        nodeList = set(dsk.keys()) - set(factor_node)
        while len(q):
            node, q = q[0], q[1:]

            topoQueue.append(node)

            append_nodes = [arg for arg in dsk[node][1:] if
                            (arg in nodeList) and (arg not in cache.keys()) and (arg not in workers_nodes)]
            nodeList = nodeList - set(append_nodes)
            q = q + append_nodes
        return topoQueue

    topo_node = topoOrder()
    nodeList = dsk.keys()

    def available_node():
        for node in topo_node:
            if dsk[node][0].mem > av_mem:
                continue
            args = dsk[node][1:]
            isAvailable = True
            for arg in args:
                if (arg in nodeList) and (arg not in cache):
                    isAvailable = False
                    break
            if isAvailable:
                return node
        return None
    if Print_limit >= Epoch:


        print(f'\nreturn: 输出一个node，表示下一步即将调用于计算的worker，如果不能调用就返回None \n{available_node()}')
        print('+'*60, '\n'*5)


    return available_node()




In [127]:
def schedule_out_func(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
    global  Print_limit, Out_Epoch
    Out_Epoch += 1

    if Print_limit >= Out_Epoch:
        print('-'*60)
        print(f'out_func Info: Epoch {Out_Epoch}\n')
        print(f'out_func的自变量与in_func完全相同。但在默认函数中，只用到了workers, cache, factor_node_dict, dsk这四个')

    factor_node = set([k for k in factor_node_dict if factor_node_dict[k] is None])

    if len(factor_node) == 0:
        return []

    workers_nodes = set([workers[k][0] for k in workers if workers[k] is not None])

    def topoOrder():
        topoQueue = []
        q = list(factor_node - set(cache.keys()) - workers_nodes)

        nodeList = set(dsk.keys()) - set(factor_node)
        while len(q):
            node, q = q[0], q[1:]

            topoQueue.append(node)

            append_nodes = [arg for arg in dsk[node][1:] if
                            (arg in nodeList) and (arg not in cache.keys()) and (arg not in workers_nodes)]
            nodeList = nodeList - set(append_nodes)
            q = q + append_nodes
        return topoQueue

    topo_node = topoOrder()

    def trash_cache():
        trash_node = []

        for node in cache:
            isTrash = True
            for tp_node in topo_node + list(workers_nodes):
                if node in dsk[tp_node][1:]:
                    isTrash = False
                    break
            if isTrash:
                trash_node.append(node)
        return trash_node
    if Print_limit >= Out_Epoch:
        print(f'return:即将被清除出内存的节点\n{trash_cache()}')
        print('-'*60, '\n'*5)


    return trash_cache()

In [128]:
from ACLScheduleTest.data import RandomDsk

rd = RandomDsk(
    mute=True, node_num=100,
    st_node_num=10,
    min_time=1, time_lv=0, time_rv=0, min_time_step=0,
    avg_mem_peek=1, mem_pv=0, mem_sv=1
)
sim = Simulator()
sim.simulate(schedule_in_func, schedule_out_func, mem_bound=9, worker_bound=10, rd=rd)

------------------------------------------------------------
out_func Info: Epoch 1

out_func的自变量与in_func完全相同。但在默认函数中，只用到了workers, cache, factor_node_dict, dsk这四个
return:即将被清除出内存的节点
[]
------------------------------------------------------------ 





++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
in_func Info: Epoch 1

workers:调度其中各个workers的状态, 是一个词典(dict)
key是worker的名字，如'worker_0'; 
value有两种，如果该worker没有正在运行, 就输出None；如果正在计算某个节点node并且已经运行了1单位时间，则输出['node', 1].
 注意，在题目假设下，每个任务都以单位时间执行，因此这里只有可能出现0
worker_0 None
worker_1 None
worker_2 None
worker_3 None
worker_4 None
worker_5 None
worker_6 None
worker_7 None
worker_8 None
worker_9 None

cache:内存存储状态，存储各个节点内用于计算的数值。也是一个dict
key是节点node的名字，value是存储的具体数值

av_mem:当前状态下可用的内存，float格式
9

factor_node_dict:因子节点的状态，key是node名字，value是输出数值，如果是None表示未被计算到
node69 None
node83 None
node80 None
node64 None
node99 None
node50 None
node73 None
node76 None
node86 None
node24 None
node94 None
node48 None
node92 None
node91 None
node89 None
node65 

({'worker_0': None,
  'worker_1': None,
  'worker_2': None,
  'worker_3': None,
  'worker_4': None,
  'worker_5': None,
  'worker_6': None,
  'worker_7': None,
  'worker_8': None,
  'worker_9': None},
 {'node0': 1.6674062571943762,
  'node1': 2.2179934520616396,
  'node4': 1.1275794170827123,
  'node7': 0.49035281641687156,
  'node2': 0.19292565842269493,
  'node8': 2.1846022197769566,
  'node9': 2.799821609816438,
  'node6': 1.5270256299057412,
  'node5': 1.7849826604310586},
 0.0,
 {'node69': None,
  'node83': None,
  'node80': None,
  'node64': None,
  'node99': None,
  'node50': None,
  'node73': None,
  'node76': None,
  'node86': None,
  'node24': None,
  'node94': None,
  'node48': None,
  'node92': None,
  'node91': None,
  'node89': None,
  'node65': None,
  'node95': None,
  'node97': None,
  'node98': None,
  'node88': None,
  'node68': None,
  'node32': None,
  'node84': None,
  'node85': None,
  'node77': None,
  'node78': None,
  'node47': None,
  'node90': None,
  'node7